In [1]:
from Simulation.EHR import generate_EHR
from Code.loaddata import open_files

# Open DATA

In [3]:
open_files('CPSC')

[[-0.02659794  0.06019023]
 [-0.0547416   0.07659802]
 [-0.02747288  0.06860666]
 ...
 [-0.03762209 -0.07251733]
 [-0.0366305  -0.06393087]
 [-0.03149757 -0.06444095]]
{'fs': 200, 'sig_len': 520088, 'n_sig': 2, 'base_date': None, 'base_time': None, 'units': ['mV', 'mV'], 'sig_name': ['I', 'II'], 'comments': ['non atrial fibrillation']}
[[-0.03442573 -0.11974417]
 [-0.04819602 -0.15112954]
 [-0.04426165 -0.11074836]
 ...
 [ 0.08885116  0.04171388]
 [ 0.0757366   0.01559273]
 [ 0.07639233 -0.03971482]]
{'fs': 200, 'sig_len': 611052, 'n_sig': 2, 'base_date': None, 'base_time': None, 'units': ['mV', 'mV'], 'sig_name': ['I', 'II'], 'comments': ['non atrial fibrillation']}
[[-0.07450193 -0.12681979]
 [-0.05866152 -0.20509137]
 [-0.06661562 -0.19174887]
 ...
 [-0.17932889  0.1742451 ]
 [-0.16177111  0.20205298]
 [-0.16177111  0.24029707]]
{'fs': 200, 'sig_len': 30483, 'n_sig': 2, 'base_date': None, 'base_time': None, 'units': ['mV', 'mV'], 'sig_name': ['I', 'II'], 'comments': ['paroxysmal atr

'10 done'

# Generate EHR


In [3]:
df_EHR = generate_EHR('CPSC')

In [4]:
df_EHR.head(20)

,Age,Gender,BMI,ASA,Drugs,Alcohol,Smoking,Blood loss,Urine loss,General Fluid Loss
0,80,Female,HW,1,0. None,0. None,2. Uses,720,274,226
1,77,Female,HW,6,1. Used,0. None,1. Used,944,484,53
2,80,Male,HW,6,2. Uses,1. Used,0. None,895,48,538
3,77,Male,OW,4,1. Used,0. None,1. Used,342,967,529
4,51,Female,OW,1,1. Used,1. Used,0. None,605,569,540
5,52,Female,OW,2,1. Used,0. None,0. None,630,363,943
6,52,Male,OW,2,1. Used,0. None,1. Used,719,782,412
7,61,Male,HW,5,2. Uses,1. Used,1. Used,401,348,437
8,76,Male,HW,5,0. None,1. Used,1. Used,536,702,936
9,55,Male,OW,3,1. Used,1. Used,2. Uses,416,944,801
